<a href="https://colab.research.google.com/github/jeffersonramelo/Textual-Analysis/blob/main/C5_legibilidade_ML.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Ler o arquivo CSV
import pandas as pd
df = pd.read_csv("/content/C456_ dataset legibilidade rotulados.csv")

In [2]:
#vizualizar dados
df

,terms,definitions,source,assigned_readability,flesch_reading_ease,flesch_kincaid_grade,smog_index,coleman_liau_index,automated_readability_index,dale_chall_readability_score,linsear_write_formula,gunning_fog
0,Low/No Documentation Loan,A low/no documentation loan allows a potential...,investopedia,1,22.75,15.8,0.0,16.42,17.8,11.99,18.000000,20.50
1,Combined Loan-To-Value Ratio (CLTV Ratio),The combined loan-to-value (CLTV) ratio is the...,investopedia,1,45.77,17.3,0.0,9.01,21.0,10.23,12.500000,18.70
2,Nonforfeiture Clause,A nonforfeiture (sometimes hyphenated) clause ...,investopedia,1,29.18,15.4,17.1,14.10,16.9,9.93,18.500000,15.33
3,Discretionary spending,Government spending authorized by Congress on ...,9_12_louis,1,28.50,11.5,0.0,15.11,11.3,12.86,6.500000,12.49
4,DEBT TO ASSETS,"A measure of a company’s FINANCIAL LEVERAGE, o...",palgrave,0,45.46,11.2,13.0,10.67,11.1,9.96,11.666667,10.00
...,...,...,...,...,...,...,...,...,...,...,...,...
1495,Sole Proprietorship,A sole proprietorship also referred to as a so...,investopedia,1,37.98,16.2,0.0,11.79,18.5,9.10,20.500000,16.84
1496,UCLA Anderson School of Management,UCLA Anderson School of Management is the grad...,investopedia,1,41.70,12.7,0.0,12.30,13.8,9.94,16.000000,16.97
1497,Unemployment,"@1@ In economic terms, involuntary unemployme...",sam,1,15.65,18.5,0.0,10.28,16.6,11.70,20.500000,16.67
1498,Pro Forma Invoice,A pro forma invoice is a preliminary bill of s...,investopedia,1,58.62,10.3,0.0,11.60,12.9,10.69,13.000000,12.21


In [3]:
#quantos dados positivo e negativos?
df.assigned_readability.value_counts()

1    874
0    626
Name: assigned_readability, dtype: int64

In [4]:
# Acessar os definitionsitionss contidos nas celulas, só substituir o número da celula que deseja vizualizar
celula = df.loc[0, 'definitions']

# Imprimir o conteúdo da célula
print(celula)


A low/no documentation loan allows a potential borrower to apply for a mortgage while providing little or no information regarding their employment, income, or assets. Regulation of these loans has evolved significantly since 2008, but they remain an option for some borrowers in nontraditional financial situations.


pre-processamento do definitions (stop words, misnusculas, caracteres especiais, stemização e tokenização)

In [5]:
pip install scikit-learn

Naive Bayes

In [6]:
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB, GaussianNB, BernoulliNB
from sklearn.metrics import confusion_matrix, accuracy_score
import pandas as pd
import numpy as np
from sklearn.impute import SimpleImputer

# Dividir o DataFrame em dados de treinamento e teste
X = df['definitions']
y = df['assigned_readability']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Preencher valores nulos com uma string vazia
imputer = SimpleImputer(strategy='constant', fill_value='')
X_train = pd.DataFrame(X_train, columns=['definitions'])
X_test = pd.DataFrame(X_test, columns=['definitions'])
X_train['definitions'] = imputer.fit_transform(X_train[['definitions']])
X_test['definitions'] = imputer.transform(X_test[['definitions']])

# Pré-processamento dos dados de definitions
vectorizer = CountVectorizer()
X_train = vectorizer.fit_transform(X_train['definitions'])
X_test = vectorizer.transform(X_test['definitions'])

# Inicializar os classificadores
mnb = MultinomialNB()
gnb = GaussianNB()
bnb = BernoulliNB()

# Realizar validação cruzada nos classificadores
cv_scores_mnb = cross_val_score(mnb, X_train, y_train, cv=5)
cv_scores_gnb = cross_val_score(gnb, X_train.toarray(), y_train, cv=5)
cv_scores_bnb = cross_val_score(bnb, X_train, y_train, cv=5)

# Treinar os classificadores nos dados de treinamento completos
mnb.fit(X_train, y_train)
gnb.fit(X_train.toarray(), y_train)
bnb.fit(X_train, y_train)

# Fazer previsões nos dados de teste
y_pred_mnb = mnb.predict(X_test)
y_pred_gnb = gnb.predict(X_test.toarray())
y_pred_bnb = bnb.predict(X_test)


In [7]:
# Calcular a matriz de confusão
confusion_mnb = confusion_matrix(y_test, y_pred_mnb)
print("confusion_mnb", confusion_mnb)
confusion_gnb = confusion_matrix(y_test, y_pred_gnb)
print("confusion_gnb", confusion_gnb)
confusion_bnb = confusion_matrix(y_test, y_pred_bnb)
print("confusion_bnb", confusion_bnb)

confusion_mnb [[ 97  32]
 [ 12 159]]
confusion_gnb [[ 75  54]
 [ 22 149]]
confusion_bnb [[121   8]
 [ 34 137]]


In [8]:
# Avaliar o desempenho dos classificadores com a métrica de acurácia
accuracy_mnb = accuracy_score(y_test, y_pred_mnb)
print("accuracy_mnb", accuracy_mnb)
accuracy_gnb = accuracy_score(y_test, y_pred_gnb)
print("accuracy_gnb", accuracy_gnb)
accuracy_bnb = accuracy_score(y_test, y_pred_bnb)
print("accuracy_bnb", accuracy_bnb)

accuracy_mnb 0.8533333333333334
accuracy_gnb 0.7466666666666667
accuracy_bnb 0.86


Suporte Vector Machine

In [9]:
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import confusion_matrix, accuracy_score

# Dividir o DataFrame em dados de treinamento e teste
X = df['definitions']
y = df['assigned_readability']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Pré-processamento dos dados de definitions
vectorizer = CountVectorizer()
X_train = vectorizer.fit_transform(X_train)
X_test = vectorizer.transform(X_test)

# Inicializar o classificador SVM
svm = SVC()

# Treinar o classificador SVM
svm.fit(X_train, y_train)

# Fazer previsões nos dados de teste
y_pred_svm = svm.predict(X_test)

# Gerar a matriz de confusão
confusion_mtx_svm = confusion_matrix(y_test, y_pred_svm)

print("Matriz de Confusão - SVM:")
print(confusion_mtx_svm)

# Calcular a acurácia do modelo SVM
accuracy_svm = accuracy_score(y_test, y_pred_svm)

print("Acurácia - SVM:", accuracy_svm)


Matriz de Confusão - SVM:
[[107  22]
 [ 27 144]]
Acurácia - SVM: 0.8366666666666667


Redes Neurais

Perceptron de Múltiplas Camadas (Multilayer Perceptron, MLP). O MLP é uma arquitetura de rede neural artificial que consiste em várias camadas de neurônios, incluindo uma camada de entrada, uma ou mais camadas ocultas e uma camada de saída. Cada neurônio em uma camada está conectado a todos os neurônios na camada seguinte, formando uma rede densamente conectada.

In [11]:
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import confusion_matrix, accuracy_score

# Dividir o DataFrame em dados de treinamento e teste
X = df['definitions']
y = df['assigned_readability']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Pré-processamento dos dados de definitions
vectorizer = CountVectorizer()
X_train = vectorizer.fit_transform(X_train)
X_test = vectorizer.transform(X_test)

# Inicializar o classificador MLP
mlp = MLPClassifier(hidden_layer_sizes=(100,), random_state=42)

# Treinar o classificador MLP
mlp.fit(X_train, y_train)

# Fazer previsões nos dados de teste
y_pred_mlp = mlp.predict(X_test)

# Gerar a matriz de confusão
confusion_mtx_mlp = confusion_matrix(y_test, y_pred_mlp)

print("Matriz de Confusão - MLP:")
print(confusion_mtx_mlp)

# Calcular a acurácia do modelo MLP
accuracy_mlp = accuracy_score(y_test, y_pred_mlp)

print("Acurácia - MLP:", accuracy_mlp)


Matriz de Confusão - MLP:
[[109  20]
 [ 30 141]]
Acurácia - MLP: 0.8333333333333334


Recurrent Neural Network (RNN): RNNs, em particular, as variantes como Long Short-Term Memory (LSTM) ou Gated Recurrent Unit (GRU), são amplamente utilizadas em análise textual. Essas arquiteturas são capazes de capturar dependências de contexto e sequenciais em texto, tornando-as adequadas para tarefas como classificação de sentimento, geração de texto ou tradução automática.

In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import confusion_matrix, accuracy_score
import tensorflow as tf

# Dividir o DataFrame em dados de treinamento e teste
X = df['definitions']
y = df['assigned_readability']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Pré-processamento dos dados de definitions
vectorizer = CountVectorizer()
X_train = vectorizer.fit_transform(X_train).toarray()
X_test = vectorizer.transform(X_test).toarray()

# Definir a arquitetura da RNN
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(input_dim=len(vectorizer.vocabulary_), output_dim=100, input_length=X_train.shape[1]),
    tf.keras.layers.SimpleRNN(units=64),
    tf.keras.layers.Dense(units=1, activation='sigmoid')
])

# Compilar o modelo
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# Treinar a RNN
model.fit(X_train, y_train, epochs=10, batch_size=32)

# Fazer previsões nos dados de teste
y_pred_rnn = model.predict(X_test)
y_pred_rnn = (y_pred_rnn > 0.5).astype(int)

# Gerar a matriz de confusão
confusion_mtx_rnn = confusion_matrix(y_test, y_pred_rnn)

print("Matriz de Confusão - RNN:")
print(confusion_mtx_rnn)

# Calcular a acurácia do modelo RNN
accuracy_rnn = accuracy_score(y_test, y_pred_rnn)

print("Acurácia - RNN:", accuracy_rnn)


Epoch 1/10
 5/38 [==>...........................] - ETA: 3:47 - loss: 0.6711 - accuracy: 0.6438

Convolutional Neural Network (CNN): Embora inicialmente desenvolvidas para processamento de imagens, as CNNs também podem ser aplicadas a análise textual, especialmente em tarefas como classificação de texto ou extração de características. As camadas convolucionais nas CNNs são capazes de extrair informações relevantes de n-gramas de palavras, capturando padrões locais em textos.

In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import confusion_matrix, accuracy_score
import tensorflow as tf

# Dividir o DataFrame em dados de treinamento e teste
X = df['definitions']
y = df['assigned_readability']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Pré-processamento dos dados de definitions
vectorizer = CountVectorizer()
X_train = vectorizer.fit_transform(X_train).toarray()
X_test = vectorizer.transform(X_test).toarray()

# Redimensionar os dados para a entrada da CNN (adicionar uma dimensão de canal)
X_train = np.expand_dims(X_train, axis=-1)
X_test = np.expand_dims(X_test, axis=-1)

# Definir a arquitetura da CNN
model = tf.keras.Sequential([
    tf.keras.layers.Conv1D(filters=64, kernel_size=3, activation='relu', input_shape=(X_train.shape[1], 1)),
    tf.keras.layers.MaxPooling1D(pool_size=2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(units=64, activation='relu'),
    tf.keras.layers.Dense(units=1, activation='sigmoid')
])

# Compilar o modelo
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# Treinar a CNN
model.fit(X_train, y_train, epochs=10, batch_size=32)

# Fazer previsões nos dados de teste
y_pred_cnn = model.predict(X_test)
y_pred_cnn = (y_pred_cnn > 0.5).astype(int)

# Gerar a matriz de confusão
confusion_mtx_cnn = confusion_matrix(y_test, y_pred_cnn)

print("Matriz de Confusão - CNN:")
print(confusion_mtx_cnn)

# Calcular a acurácia do modelo CNN
accuracy_cnn = accuracy_score(y_test, y_pred_cnn)

print("Acurácia - CNN:", accuracy_cnn)
